In [1]:
import pandas as pd
import numpy as np
from IPython.display import HTML
from collections import OrderedDict

In [2]:
dir_ = '../../output/stage_2_predictions/Keck_Pria_AS_Retest'
complete_df = pd.read_csv('{}/complete_prediction.csv'.format(dir_))

# Show 84 Active Compounds' Corresponding Predicted Values

In [3]:
dataframe = pd.read_excel('../../output/stage_2_predictions/Keck_LC4_export.xlsx')

supplier_id = dataframe['Supplier ID'].tolist()
failed_id = ['F0401-0050', 'F2964-1411', 'F2964-1523']
inhibits = dataframe[
    'PriA-SSB AS, normalized for plate and edge effects, correct plate map: % inhibition Alpha, normalized (%)'].tolist()

positive_enumerate = filter(lambda x: x[1] >= 35 and supplier_id[x[0]] not in failed_id, enumerate(inhibits))
positive_idx = map(lambda x: x[0], positive_enumerate)

In [4]:
model_names = complete_df.columns[3:]

In [5]:
columns = ['inhibition']
columns.extend(model_names)
HTML(complete_df[complete_df['label']>0][columns].to_html())

,inhibition,Baseline,ConsensusDocking_efr1_opt,ConsensusDocking_max,ConsensusDocking_mean,ConsensusDocking_median,ConsensusDocking_rocauc_opt,Docking_ad4,Docking_dock6,Docking_fred,Docking_hybrid,Docking_plants,Docking_rdockint,Docking_rdocktot,Docking_smina,Docking_surflex,IRV_a,IRV_b,IRV_c,IRV_d,IRV_e,LSTM_a,LSTM_b,MultiClassification_a,MultiClassification_b,RandomForest_a,RandomForest_b,RandomForest_c,RandomForest_d,RandomForest_e,RandomForest_f,RandomForest_g,RandomForest_h,SingleClassification_a,SingleClassification_b,SingleRegression_a,SingleRegression_b
456,93.622204,0.37,0.032070,9.558674,8.300441,7.915640,0.035326,4.65,29.266851,7.039242,-0.235210,58.0847,14.804,35.913,5.871,3.82,0.500282,0.499869,0.499749,0.501856,0.507566,0.002003,0.041430,1.306898e-01,1.417417e-02,0.63800,0.627500,0.633875,0.63925,0.629500,0.75200,0.66100,0.652000,0.999364,0.745777,111.501968,115.448441
483,83.123806,0.63,0.026200,11.627432,10.401617,10.348252,0.052207,6.83,49.549496,10.348237,-0.216938,96.0663,16.759,38.103,8.955,7.17,0.500282,0.499869,0.504866,0.505282,0.511596,0.002003,0.041506,8.293058e-05,5.710558e-05,0.26250,0.263750,0.261125,0.20825,0.209625,0.28625,0.16450,0.160625,0.000011,0.013834,-3.408564,0.307884
1239,72.063678,0.54,0.020902,10.643016,9.375816,9.086127,0.026928,6.72,41.775131,8.748974,0.820223,81.6115,14.474,34.089,8.930,3.36,0.500282,0.522985,0.545548,0.543723,0.554211,0.002003,0.041435,2.490977e-01,2.412796e-01,0.04125,0.039250,0.040313,0.04925,0.054250,0.42625,0.11950,0.118250,0.165573,0.206531,29.097578,85.389671
1878,89.493228,0.32,0.027703,12.084562,10.107620,10.038560,0.039552,6.65,40.496170,11.273830,2.226941,79.9549,16.883,37.892,8.525,4.85,0.710454,0.575938,0.595639,0.592230,0.601834,0.001995,0.041460,2.596361e-04,4.096619e-03,0.62775,0.629375,0.631313,0.55125,0.555625,0.77675,0.61225,0.606500,0.000062,0.025602,4.825106,13.609948
2179,44.850520,0.63,0.136960,13.457431,10.838761,10.761808,0.154201,7.66,43.351162,9.481766,-0.060435,89.0793,27.440,47.976,7.494,8.44,0.500282,0.499869,0.499749,0.500670,0.505504,0.001990,0.041254,8.081179e-09,4.435486e-07,0.23500,0.231750,0.230875,0.23600,0.239375,0.01825,0.23175,0.233625,0.000035,0.012207,-5.790788,-4.602567
2309,93.499335,0.33,0.100197,12.998991,11.054573,10.995590,0.161404,6.55,48.574375,12.967777,1.797404,88.8323,20.722,41.832,9.279,6.81,0.500282,0.511202,0.533921,0.532971,0.542782,0.002001,0.041641,3.407333e-05,1.647835e-04,0.01600,0.017875,0.016125,0.01975,0.021000,0.08550,0.05475,0.052625,0.000329,0.109288,6.454531,12.292480
3179,71.993342,0.73,0.014772,11.898092,10.060977,9.957698,0.025295,6.20,42.910912,10.002242,0.058531,74.8190,21.441,44.496,7.839,6.04,0.696237,0.562904,0.574395,0.574022,0.581741,0.002005,0.041476,6.132637e-02,3.185447e-02,0.09375,0.098250,0.101437,0.16750,0.161875,0.66325,0.33125,0.325750,0.042266,0.161456,15.114231,46.100803
3327,69.353268,0.33,0.272675,13.327268,11.368801,11.679024,0.300917,9.49,61.445744,8.386213,-2.044100,92.4668,22.579,47.814,10.382,8.16,0.500282,0.499869,0.499749,0.500670,0.505504,0.002000,0.041377,6.992787e-08,2.628636e-07,0.00050,0.000375,0.000500,0.00100,0.000750,0.00000,0.00150,0.001000,0.000004,0.016198,-1.251667,-3.816286
3335,55.856874,0.28,0.009858,10.433755,9.889037,9.891108,0.017578,6.27,43.284203,10.395682,-0.494275,85.9378,17.885,38.858,8.215,5.64,0.500282,0.499869,0.499749,0.500670,0.505504,0.002002,0.041470,2.231181e-05,6.569731e-06,0.00300,0.002000,0.001687,0.00225,0.002000,0.00000,0.00125,0.000875,0.000021,0.023615,-2.002434,-3.494677
3419,67.383500,0.36,0.100206,12.065390,10.921904,11.003480,0.165604,6.51,51.692665,11.061377,0.439846,93.1481,20.617,42.722,8.556,8.61,0.500282,0.499869,0.499749,0.500670,0.505564,0.002003,0.041378,4.826655e-05,9.704479e-06,0.00025,0.000125,0.000063,0.00025,0.000125,0.00000,0.00050,0.000375,0.000018,0.014361,-3.995442,0.628636


## Show 84 Active Compounds' Corresponding Predicted Rank

In [6]:
rank_df = pd.read_csv('{}/complete_rank.csv'.format(dir_))

In [7]:
HTML(rank_df[rank_df['label']>0].to_html())

,molecule id,label,inhibition,Baseline,ConsensusDocking_efr1_opt,ConsensusDocking_max,ConsensusDocking_mean,ConsensusDocking_median,ConsensusDocking_rocauc_opt,Docking_ad4,Docking_dock6,Docking_fred,Docking_hybrid,Docking_plants,Docking_rdockint,Docking_rdocktot,Docking_smina,Docking_surflex,IRV_a,IRV_b,IRV_c,IRV_d,IRV_e,LSTM_a,LSTM_b,MultiClassification_a,MultiClassification_b,RandomForest_a,RandomForest_b,RandomForest_c,RandomForest_d,RandomForest_e,RandomForest_f,RandomForest_g,RandomForest_h,SingleClassification_a,SingleClassification_b,SingleRegression_a,SingleRegression_b
456,F0447-0233,1,93.622204,867,10971,24586,24693,24931,15037,23333,24765,24585,17758,24832,21943,19947,24950,24576,12691,12735,12815,482,479,3568,7077,6,13,3,5,4,3,4,5,3,3,3,3,2,1
483,F0472-0070,1,83.123806,15,13461,8213,8061,9293,10033,11030,7400,10116,17601,1428,16679,13344,6978,5892,12691,12735,326,334,336,2197,1880,1710,1744,10,10,10,13,13,40,23,24,24198,11710,9516,4541
1239,F0696-4000,1,72.063678,73,16599,20530,21882,22898,18729,12155,20739,21629,7077,14124,22518,23177,7214,24985,12691,115,78,91,79,3096,6559,4,3,80,80,80,76,67,22,33,33,13,17,61,6
1878,F1031-0163,1,89.493228,1938,12726,3754,13302,15339,13543,12895,21905,2710,552,15804,16284,14005,11721,22002,74,13,15,16,16,18857,4336,621,38,5,4,5,5,5,2,5,5,4459,2034,566,372
2179,F1243-0026,1,44.850520,15,1358,206,2204,3330,1859,3639,18771,17752,16161,6007,25,167,21375,832,12691,12735,12815,12980,13348,23391,22263,24495,18653,11,12,12,10,10,495,13,13,12564,14359,18025,13008
2309,F1310-0123,1,93.499335,1612,2397,653,895,1528,1700,13875,9185,82,1390,6248,3969,3298,4241,8496,12691,174,116,117,113,10181,297,3471,597,159,148,157,137,131,134,69,71,159,43,437,435
3179,F1835-0132,1,71.993342,2,21805,5385,14116,16487,19636,17046,19364,14233,14781,20351,2652,844,18983,14819,80,20,26,26,27,227,3150,13,8,42,37,36,17,19,11,8,8,18,27,152,36
3327,F1883-1660,1,69.353268,1612,266,293,206,75,340,35,90,22745,24594,3208,1279,181,311,1377,12691,12735,12815,12980,13348,14173,14068,22730,20108,5745,6829,5454,4123,5270,14809,2108,3139,25235,8365,3636,11402
3335,F1885-0026,1,55.856874,4398,25198,22122,16733,17103,24067,16446,18879,9593,19743,9151,12648,10788,15246,17807,12691,12735,12815,12980,13348,7208,3605,4595,8079,961,1506,1762,1585,1713,14809,2595,3652,19841,2742,5161,10780
3419,F1899-1464,1,67.383500,995,2395,3923,1564,1481,1601,14249,4165,3787,10583,2794,4213,2156,11316,586,12691,12735,12815,12980,900,3662,13975,2677,6542,8945,11991,14993,11983,15661,14809,6685,8045,21519,10935,11676,4191


# Show n-hits in top 250

In [8]:
model_names = rank_df.columns[3:]

# Clustering Analysis

In [9]:
clustering_df = pd.read_excel('{}/clustering/lc4_clusters.xlsx'.format(dir_))
clustering_names = clustering_df.columns[2:].tolist()

In [10]:
merged_clustering_df = pd.merge(rank_df, clustering_df, left_on=['molecule id'], right_on='Batch')

## Clustering Number for All 84 Actives

In [11]:
for clustering_name in clustering_names:
    clustering_ids = merged_clustering_df[clustering_name].loc[positive_idx].tolist()
    clustering_num = len(set(clustering_ids))
    print clustering_name, ':\t', clustering_num

SIM_Cluster :	40
MCS_Cluster :	40


## Clustering Number for All 84 Actives

In [12]:
model_mapping_to_hits = []
model_mapping_to_clustering = {}
for clustering_name in clustering_names:
    model_mapping_to_clustering[clustering_name] = []

for (idx, model_name) in enumerate(model_names):
    
    if 'ensemble' in model_name:
        order = rank_df[model_name].rank().loc[positive_idx].tolist()
        order = np.array(order)
        order = order.astype(int)
        continue
    else:
        order = rank_df[model_name].loc[positive_idx]
    
    order = rank_df[model_name].loc[positive_idx]
        
    hits_id = filter(lambda x: x<=250, order)
    hits = len(hits_id)
    model_mapping_to_hits.append(len(hits_id))

    for clustering_name in clustering_names:
        clustering_ids = set(merged_clustering_df[clustering_name].loc[hits_id].tolist())
        clustering_num = len(clustering_ids)
        model_mapping_to_clustering[clustering_name].append(clustering_num)

In [13]:
model_mapping_to_hits = []
model_mapping_to_clustering = {}
for clustering_name in clustering_names:
    model_mapping_to_clustering[clustering_name] = []

for (idx, model_name) in enumerate(model_names):
    order = rank_df[model_name].loc[positive_idx]
    hits_id_tuple = filter(lambda x: x[1] < 250, order.iteritems())
    hits_id = [x[0] for x in hits_id_tuple]
    model_mapping_to_hits.append(len(hits_id))

    for clustering_name in clustering_names:
        clustering_ids = set(merged_clustering_df[clustering_name].loc[hits_id].tolist())
        clustering_num = len(clustering_ids)
        model_mapping_to_clustering[clustering_name].append(clustering_num)

In [14]:
hit_and_clustering_df = pd.DataFrame({'model_names':model_names})
hit_and_clustering_df['n-hits'] = model_mapping_to_hits
for clustering_name in clustering_names:
    hit_and_clustering_df[clustering_name] = model_mapping_to_clustering[clustering_name]
    
hit_and_clustering_df

,model_names,n-hits,SIM_Cluster,MCS_Cluster
0,Baseline,33,16,18
1,ConsensusDocking_efr1_opt,0,0,0
2,ConsensusDocking_max,2,2,2
3,ConsensusDocking_mean,1,1,1
4,ConsensusDocking_median,2,2,1
5,ConsensusDocking_rocauc_opt,0,0,0
6,Docking_ad4,6,5,5
7,Docking_dock6,3,3,2
8,Docking_fred,2,2,2
9,Docking_hybrid,2,2,2
